In [1]:
bold = !ls /opt/animalfmritools/animalfmritools/data/MouseConcussion/bids/derivatives/bold_preproc/sub-AppTauApoe4141M2/ses-s1/func/*nii.gz
confounds = !ls /opt/animalfmritools/animalfmritools/data/MouseConcussion/bids/derivatives/bold_preproc/sub-AppTauApoe4141M2/ses-s1/func/*tsv

b, c = bold[0], confounds[0]

In [5]:
import pandas as pd
import nibabel as nib

confound_df = pd.read_csv(c, sep="\t")

confound_cols = []
for i in confound_df.columns:
    if i == "csf" or i == "white_matter":
        confound_cols.append(i)
    if i.startswith("trans_") or i.startswith("rot_") or i.startswith("cosine"):
        confound_cols.append(i)


regressors = confound_df.loc[1:][confound_cols].values
print(regressors.shape)
func_img = nib.load(b).slicer[:, :, :, 1:]
print(func_img.shape)
print(confound_cols)

(599, 39)
(33, 49, 16, 599)
['csf', 'white_matter', 'cosine00', 'cosine01', 'cosine02', 'cosine03', 'cosine04', 'cosine05', 'cosine06', 'cosine07', 'cosine08', 'cosine09', 'cosine10', 'cosine11', 'cosine12', 'trans_x', 'trans_x_derivative1', 'trans_x_derivative1_power2', 'trans_x_power2', 'trans_y', 'trans_y_derivative1', 'trans_y_power2', 'trans_y_derivative1_power2', 'trans_z', 'trans_z_derivative1', 'trans_z_derivative1_power2', 'trans_z_power2', 'rot_x', 'rot_x_derivative1', 'rot_x_derivative1_power2', 'rot_x_power2', 'rot_y', 'rot_y_derivative1', 'rot_y_derivative1_power2', 'rot_y_power2', 'rot_z', 'rot_z_derivative1', 'rot_z_derivative1_power2', 'rot_z_power2']


In [3]:
import numpy as np
from nilearn.image import clean_img

In [4]:
TR = func_img.header.get_zooms()[-1]
denoised_img = clean_img(
    func_img,
    confounds=regressors,
    detrend=False,
    standardize=True,
    t_r=TR,
)

denoised_data = denoised_img.get_fdata()
mean_data = func_img.get_fdata().mean(axis=3)[:, :, :, np.newaxis]
denoised_data += mean_data
denoised_img = nib.Nifti1Image(
    denoised_data, affine=denoised_img.affine, header=denoised_img.header
)
nib.save(denoised_img, "denoised.nii.gz")

/usr/local/lib/python3.10/dist-packages/nilearn/image/image.py:1271: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  data = signal.clean(
